In [1]:
from geographiclib.geodesic import Geodesic
import heapq

def compute_geodesic_points(start_point, end_point, num_points):
    geod = Geodesic.WGS84
    geodesic_line = geod.InverseLine(start_point[0], start_point[1], end_point[0], end_point[1])
    points = []
    for i in range(num_points + 1):
        interp = geodesic_line.Position(i * geodesic_line.s13 / num_points)
        points.append((interp['lat2'], interp['lon2']))
    return points

def heuristic_distance(point, goal):
    dx = goal[0] - point[0]
    dy = goal[1] - point[1]
    return ((dx ** 2) + (dy ** 2)) ** 0.5

def astar(start, goal, obstacles, num_points):
    geod = Geodesic.WGS84
    frontier = [(0, start)]
    came_from = {}
    cost_so_far = {start: 0}
    while frontier:
        _, current = heapq.heappop(frontier)
        if current == goal:
            break
        for next_point in compute_geodesic_points(current, goal, num_points):
            if next_point not in obstacles:
                new_cost = cost_so_far[current] + heuristic_distance(current, next_point)
                if next_point not in cost_so_far or new_cost < cost_so_far[next_point]:
                    cost_so_far[next_point] = new_cost
                    priority = new_cost + heuristic_distance(next_point, goal)
                    heapq.heappush(frontier, (priority, next_point))
                    came_from[next_point] = current
    path = [goal]
    while goal != start:
        goal = came_from[goal]
        path.append(goal)
    path.reverse()
    return path

start_point = (40.7128, -74.0060) 
end_point = (34.0522, -118.2437)  

obstacle_points = [
    (34.05219999999999, -118.2437)   
]

num_points = 10
shortest_path = compute_geodesic_points(start_point, end_point, num_points)
if any(point in obstacle_points for point in shortest_path):
    print("Obstacles detected in shortest path by GeographicLib.")
    print("Finding alternative path using A* algorithm.")
    for delta_lat in [-1, 0, 1]:
        for delta_lon in [-1, 0, 1]:
            adjusted_start_point = (start_point[0] + delta_lat, start_point[1] + delta_lon)
            adjusted_end_point = (end_point[0] + delta_lat, end_point[1] + delta_lon)
            alternative_path = astar(adjusted_start_point, adjusted_end_point, obstacle_points, num_points)
            if all(point not in obstacle_points for point in alternative_path):
                print("Alternative path without obstacles:")
                print(alternative_path)
                break
        else:
            continue
        break
else:
    print("Shortest path by GeographicLib (no obstacles):")
    print(shortest_path)

Shortest path by GeographicLib (no obstacles):
[(40.7128, -74.006), (40.84967033430713, -78.67586478629836), (40.79709517970596, -83.35164463056793), (40.555715879307975, -88.00686795595834), (40.128443476304135, -92.61605962755559), (39.52029944740094, -97.15591464581742), (38.738153336520206, -101.60623958380341), (37.79038880558262, -105.95058555957014), (36.68653433340621, -110.17655028608988), (35.436893089900316, -114.27577549039238), (34.0522, -118.24370000000002)]
